# Lecture 13 - Gaussian Mixture Models

In [8]:
from scipy.stats import multivariate_normal
import numpy as np
import numpy.random as npr

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-colorblind')

from IPython.display import Image

# Gaussian Mixture Models

A **Gaussian Mixture Model** or **GMM** is a probabilistic model that assumes a data likelihood to be a weighted sum of Gaussian distributions with unknown parameters.

$$p(\mathbf{x}) = \sum_{k=1}^K \pi_k N(\mathbf{x}|\mathbf{\mu}_k,\Sigma_k)$$

where $0 \leq \pi_k \leq 1$ and $\sum_{k=1}^K \pi_k = 1$.

* When standard distributions (such as Gamma, Exponential, Gaussian, etc.) are not sufficient to characterize a *complicated* data likelihood, we can instead characterize it as the sum of weighted Gaussians distributions

* Another way that GMMs are most commonly used for is to partition data in subgroups

## Data Likelihood as a Gaussian Mixture Model

* GMMs can be used to learn a complex distribution that represent a dataset. Thus, it can be used within the probabilistic generative classifier framework to model complex data likelihoods.

* GMMs are also commonly used for **clustering**. Here a GMM is fit to a dataset with the goal of partitioning it into clusters.

**<font color=blue>Step 1</font>**

Describe the **observed data likelihood**, $\mathcal{L}^o$. As seen last class:

$$\mathcal{L}^0 = \prod_{i=1}^N \sum_{k=1}^K \pi_k \mathcal{N}(x_i|\mu_k, \Sigma_k)$$

**<font color=blue>Step 2</font>**

Describe the log-likelihood function:

\begin{align}
\mathcal{L} &= \ln\left(\prod_{i=1}^N \sum_{k=1}^K \pi_k \mathcal{N}(x_i|\mu_k, \Sigma_k)\right)\\
\iff \mathcal{L} &= \sum_{i=1}^N \ln \left( \sum_{k=1}^K \pi_k \mathcal{N}(x_i|\mu_k, \Sigma_k) \right)
\end{align}

**<font color=blue>Step 3</font>**

Optimize for the parameters $\Theta=\{\pi_k, \mu_k,\Sigma_k\}_{k=1}^K$

$$\frac{\partial \mathcal{L}}{\partial \mu_k} = 0, \frac{\partial \mathcal{L}}{\partial \Sigma_k} = 0, \text{ and }, \frac{\partial \mathcal{L}}{\partial \pi_k} = 0$$

but this is a difficult problem to maximize!

* A common approach for estimating the parameters of a GMM given a data set is by using the **Expectation-Maximization (EM) algorithm**.

# Expectation-Maximization (EM) algorithm

## Hidden Latent Variables

The **Expectation-Maximization** or **EM** algorithm is used to find the Maximum Likelihood Estimators (MLE) (or MAP estimators) for model parameters when data is incomplete, has missing data points, or has unobserved (hidden) latent variables (such as the case of censored data). 

* For all of these cases, the MLE optimization is very difficult to obtain by simply taking the derivative and solve for the parameters.

**<font color=orange>Step 1</font>**
The first step of EM is to characterized the observed likelihood $\mathcal{L}^0$.

**<font color=orange>Step 2</font>**
Introduce *hidden latent variables* (also referred to as *hidden variables*) that simplify the observed data likelihood, $\mathcal{L}$.

**<font color=orange>Step 3</font>**
Use the hidden variables to define the *complete likelihood* $\mathcal{L}^c$.

With this, we build the EM optimization function:

$$\arg_{z,\Theta} \max Q(\Theta,\Theta^t)$$

where 

$$Q(\Theta,\Theta^t) = E[\ln(\mathcal{L^c})|X,\Theta^t],$$

$E[\bullet]$ denotes expected value and $t$ denotes *iteration*. At $t=0$, we start with random values for the parameters $\Theta$.

Once we have this, the EM algorithm will alternate between E-step and M-step:

1. **<font color=blue>E-step</font> (Expectation step)** Estimate the hidden variables. While holding $\Theta$ fixed, find the variables $z$ that maximize $E[\ln(\mathcal{L^c})]$.

2. **<font color=blue>M-step</font> (Maximization step)** Estimate the parameters of the complete data likelihood $\mathcal{L}^c$. While holding the newly found variables $z$, find the best values for the parameters $\Theta$ that maximize $E[\ln(\mathcal{L^c})]$.

and it keeps iterating between E-step and M-step until convergence or until a certain number of iterations is reached.

* EM is an **alternating optimization** algorithm, as it alternates between E-step and M-step in order to find the hidden variables and best set of parameters, respectively.
    * In the first step ($t=0$), EM will start with a random guess for the value of the parameters $\Theta$ in order to perform the E-step.
    * What is the problem with alternating optimization algorithms in general?

* EM is a general algorithm that can be applied to a variety of problems (not just the examples we are learning today). 

* EM is heavily tied with Maximum Likelihood Estimation (MLE). It is commonly used to simplify difficult MLE problems.
    * It was originally introduced by Dempster, Laird, and Rubin in 1977 in a paper called "Maximum Likelihood from Incomplete Data via the EM Algorithm".

## Optimizing GMM with the EM Algorithm

The observed data likelihood for a Gaussian Mixture Model (GMM) is

$$\mathcal{L}^0 = \prod_{i=1}^N \sum_{k=1}^K \pi_k \mathcal{N}(x_i|\mu_k, \Sigma_k)$$

**What hidden variables can we add to simplify this problem?**

* In this example, a hidden variable can be the label of the Gaussian from which $x_i$ was drawn from.

$$z_i: \text{label of the Gaussian from which $x_i$ was drawn from}$$

If we know $z_i$, then for each data point we know which Gaussian it was drawn from along with its respective parameter $\mu_{z_i}$ and $\Sigma_{z_i}$ and its respective weight $\pi_{z_i}$. So, each data point would had been drawn from $\pi_{z_i}N(x_i|\mu_{z_i}, \Sigma_{z_i})$. 

Then, assuming we have $\{z_i\}_{i=1}^N$, we can write the complete data likelihood:

$$\mathcal{L}^c = \prod_{i=1}^N \pi_{z_i}\mathcal{N}(x_i|\mathbf{\mu}_{z_i},\Sigma_{z_i})$$

Now we can iterate between the **E-step** and **M-step** of the EM algorithm until we find convergence or we have reached a threshold for a number of iterations.

# Optimization Function

We can now extent the optimization function:

\begin{align}
Q(\Theta,\Theta^t) &= E[\ln(\mathcal{L^c})|X,\Theta^t] \\
&= \sum_{\mathbf{z}} \ln(\mathcal{L^c}) P(z|X,\Theta^t) \\
&= \sum_{z_i=1}^K \ln(\mathcal{L^c}) P(\mathbf{z}_i|\mathbf{x}_i,\Theta^t)
\end{align}

**<font color=blue>E-step</font>**

In order to complete the E-STEP, we need to know how to compute $P(\mathbf{z}_i|\mathbf{x}_i,\Theta^t)$. 

* This is the posterior probability of the label $z_i$ for data sample $x_i$.

* So we want to assign the label $z_i$ to the data sample $x_i$ for which the posterior probability is maximized (just like in Naive Bayes classification).

Recall from **Bayes' Rule**: for two non-empty events $A$ and $B$, $P(A|B) = \frac{P(B|A)P(A)}{P(B)}$. We can use this theorem to rewrite $P(\mathbf{z}_i|\mathbf{x}_i,\Theta^t)$:

\begin{align}
P(\mathbf{z}_i|\mathbf{x}_i,\Theta^t) &= \frac{P(\mathbf{x}_i|\mathbf{z}_i,\Theta^t)P(\mathbf{z}_i|\Theta^t)}{P(\mathbf{x}_i|\Theta^t)} \\
&= \frac{P(\mathbf{x}_i|\mathbf{\mu}_{z_i}^t, \Sigma_{z_i}^t) \pi_{z_i}^t}{\sum_{z_i=1}^K \pi_{z_i}^t P(\mathbf{x}_i|\mathbf{\mu}_{z_i}^t, \Sigma_{z_i}^t)} \\
&= C_{ik}
\end{align}

This is called the **memberships** or **responsabilities** matrix, which contains the label assignment for point $x_i$ in each Gaussian component $k.

* In the E-STEP we estimate the membership matrix $C_{ik} = P(\mathbf{z}_i|\mathbf{x}_i,\Theta^t)$. This matrix is of size $N\times K$ that contains the likelihoods of each point belonging in each one of the Gaussians.

* A good check when implementing this matrix is to make sure that the sum of the rows are equal to 1!

**This completes the Expectation step (E-step) in EM**. Now, we derive the update equations for the parameters $\Theta=\{\pi_k,\mu_k,\Sigma_k\}_{k=1}^K$  in the Maximization step.

**<font color=blue>M-step</font>**

In the **M-step**, we are going to use (and hold constant) the membership matrix $C_{ik}$ we learned from the E-step.

We will know estimate the new set of parameters $\Theta=\{\pi_k,\mu_k,\Sigma_k\}_{k=1}^K$ that maximize $Q(\Theta,\Theta^t)$, i.e.

$$\arg_{\mathbf{\Theta}}\max Q(\Theta,\Theta^t)$$

Without loss of generality, let's assume that the covariance matrices are isotropic: $\Sigma_k = \sigma_k^2\mathbf{I}$, then we can rewrite it as:

\begin{align}
Q(\Theta, \Theta^t) &= \sum_{z_i=1}^K \ln(\mathcal{L^c}) P(\mathbf{z}_i|\mathbf{x}_i,\Theta^t)\\
&= \sum_{z_i=1}^K \ln\left(\prod_{i=1}^N \pi_{z_i}\mathcal{N}(x_i|\mathbf{\mu}_{z_i},\Sigma_{z_i})\right) P(\mathbf{z}_i|\mathbf{x}_i,\Theta^t) \\
&= \sum_{k=1}^K \ln\left(\prod_{i=1}^N \pi_k\mathcal{N}(x_i|\mathbf{\mu}_k,\Sigma_k)\right) P(\mathbf{z}_i =k|\mathbf{x}_i,\Theta^t) \\
&= \sum_{k=1}^K \sum_{i=1}^N \left(\ln(\pi_k) + \ln\left(\mathcal{N}(x_i|\mathbf{\mu}_k,\Sigma_k)\right) \right) C_{ik}\\
&= \sum_{k=1}^K \sum_{i=1}^N \left( \ln(\pi_k) -\frac{1}{2}\ln(2\pi) -\frac{1}{2}\ln(\sigma_k^2) - \frac{1}{2\sigma^2_k}\Vert \mathbf{x}_i -\mu_k\Vert_2^2 \right) C_{ik}
\end{align}

So, now we want to solve:

\begin{align}
\frac{\partial Q(\Theta,\Theta^t)}{\partial \mu_k} &= 0 \\
\frac{\partial Q(\Theta,\Theta^t)}{\partial \sigma_k^2} &= 0 \\
\frac{\partial Q(\Theta,\Theta^t)}{\partial \pi_k} &= 0 
\end{align}

### Solving for $\mu_k$

\begin{align}
0 &= \frac{\partial Q(\Theta,\Theta^t)}{\partial \mu_k} \\
0 &= \sum_{i=1}^N \left(\frac{1}{\sigma^2_k}(\mathbf{x}_i -\mu_k)\right) C_{ik}\\
0 &= \sum_{i=1}^N (\mathbf{x}_i -\mu_k) C_{ik} \\
0 &= \sum_{i=1}^N \mathbf{x}_i C_{ik} - \sum_{i=1}^N \mu_k C_{ik} \\
\sum_{i=1}^N \mathbf{x}_i C_{ik} &= \mu_k \sum_{i=1}^N C_{ik} \\
\mu_k &= \frac{\sum_{i=1}^N \mathbf{x}_i C_{ik}}{\sum_{i=1}^N C_{ik}}
\end{align}


* We can interpret this as a **weighted mean**, where the weights represent the likelihood of each point belonging to cluster $k$.

### Solving for $\sigma_k^2$

\begin{align}
0 &= \frac{\partial Q(\Theta,\Theta^t)}{\partial \sigma_k^2} \\
0 &= \sum_{i=1}^N \left( -\frac{1}{2\sigma_k^2} + \frac{2}{(2\sigma_k^2)^2} \Vert x_i-\mu_k\Vert_2^2 \right) C_{ik}\\
\frac{1}{2\sigma_k^2}\sum_{i=1}^N C_{ik} &= \frac{2}{(2\sigma_k^2)^2} \sum_{i=1}^N C_{ik} \Vert x_i-\mu_k\Vert_2^2 \\
\sigma^2_k\sum_{i=1}^N C_{ik} &= \sum_{i=1}^N C_{ik} \Vert x_i-\mu_k\Vert_2^2 \\
\sigma^2_k &= \frac{\sum_{i=1}^N C_{ik} \Vert x_i-\mu_k\Vert_2^2}{\sum_{i=1}^N C_{ik}}
\end{align}

* Again, assuming that the covariance of each cluster is isotropic, that is, $\Sigma_k = \sigma_k^2\mathbf{I}$.

* For the variance, this can be interpreted as how much each data point contributes to the k-th cluster.

### Solving for $\pi_k$

For $\pi_k$ we have the contraint that $\sum_{k=1}^K \pi_k =1$. Then we need to incorporate it into $Q(\Theta, \Theta^t)$ using Lagrange Multipliers.

$$Q_{\pi}(\Theta,\Theta^t) = Q(\Theta,\Theta^t) + \lambda \left(1 - \sum_{k=1}^K\pi_k\right)$$

where $\lambda$ is called the Lagrange multiplier.

To be finished next class.